<a href="https://colab.research.google.com/github/PurJiri/colab/blob/main/V%C3%ADtejte_ve_slu%C5%BEb%C4%9B_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()


KeyboardInterrupt



In [2]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

In [3]:
data = np.genfromtxt("0_and_1.csv", delimiter=',', skip_header=1)
print(data)

# Extract the target variable Y and reshape it
Y = data[:, 0] # copy the first column - the "label"
Y = Y.reshape(1, -1)

# Remove the label column from X and transpose the matrix
X = data[:, 1:].T

FileNotFoundError: 0_and_1.csv not found.

In [ ]:
print("X", X.shape)  # (28x28=784 columns, 8816 rows)
print("Y", Y.shape)  # (1 column, 8816 rows)

In [ ]:
fig = plt.figure(figsize=(8, 8))
columns = 4
rows = 5

# Loop through and display each image
for i in range(1, columns * rows + 1):
    X1 = X[:, i-1]  # Extract the i-th column (i-th image)
    X1 = X1.reshape(28, 28)  # Reshape the flattened image to (28, 28)
    fig.add_subplot(rows, columns, i)  # Add a subplot to the figure
    plt.imshow(X1, cmap='gray')  # Display the image using a grayscale colormap

plt.show()

In [ ]:

def sigmoid(x, derivative=False):

    sig = 1 / (1 + np.exp(-x))
    if derivative:
        return sig * (1 - sig)
    else:
        return sig

In [ ]:
print(sigmoid(0))                   # Output: 0.5
print(sigmoid(0, derivative=True))  # Output: 0.25

In [ ]:
def network_architecture(X, Y):
    # Number of nodes in the input layer
    n_x = X.shape[0]
    # Number of nodes in the hidden layer
    n_h = 10
    # Number of nodes in the output layer
    n_y = Y.shape[0]
    return (n_x, n_h, n_y)

n_x, n_h, n_y = network_architecture(X, Y)

# Print the number of nodes in each layer
print("Number of nodes in input layer (n_x):", n_x) # 784
print("Number of nodes in hidden layer (n_h):", n_h) # 10
print("Number of nodes in output layer (n_y):", n_y) # 1

In [ ]:
def define_network_parameters(n_x, n_h, n_y):
    W1 = np.random.randn(n_h, n_x) * 0.01  # Random initialization with small values
    b1 = np.zeros((n_h, 1))  # Initialize biases with zeros

    W2 = np.random.randn(n_y, n_h) * 0.01  # Random initialization with small values
    b2 = np.zeros((n_y, 1))  # Initialize biases with zeros

    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}

    return parameters

In [ ]:
n_x, n_h, n_y = network_architecture(X, Y)

params = define_network_parameters(n_x, n_h, n_y)

print("W1", params['W1'].shape) # W1 (10, 784)
print("b1", params['b1'].shape) # b1 (10, 8816)
print("W2", params['W2'].shape) # W2 (1, 10) - vector of 10 elements, same as the number of hidden nodes
print("b2", params['b2'].shape) # b2 (1, 1) - single element, same as the number of output nodes


In [ ]:
def forward_propagation(X, params):
    W1 = params['W1']
    b1 = params['b1']
    W2 = params['W2']
    b2 = params['b2']
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)  # Activation function

    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)

    return {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}

In [ ]:
refor = forward_propagation(X, params)

print("Z1", refor['Z1'].shape) # Z1 (10, 8816)
print("A1", refor['A1'].shape) # A1 (10, 8816)
print("Z2", refor['Z2'].shape) # Z2 (1, 8816)
print("A2", refor['A2'].shape) # A2 (1, 8816)

In [ ]:
def backward_propagation(params, activations, X, Y):

    A2 = activations['A2']
    Z2 = activations['Z2']
    A1 = activations['A1']
    Z1 = activations['Z1']

    W2 = params['W2']

    m = X.shape[1]  # Number of training examples

    # Output layer
    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T) / m
    db2 = dZ2.sum(axis=1, keepdims=True) / m

    dZ1 = np.dot(W2.T, dZ2) * sigmoid(Z1, derivative=True)
    dW1 = np.dot(dZ1, X.T) / m
    db1 = dZ1.sum(axis=1, keepdims=True) / m

    return {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}

In [ ]:
def update_parameters(params, derivatives, alpha=0.1):

    params['W1'] -= alpha * derivatives['dW1']
    params['b1'] -= alpha * derivatives['db1']
    params['W2'] -= alpha * derivatives['dW2']
    params['b2'] -= alpha * derivatives['db2']
    return params

In [ ]:
derivatives = backward_propagation(params, refor, X, Y)

params = update_parameters(params, derivatives)

print(params)

In [ ]:
def compute_error(Predicted, Actual):
    return np.mean((Predicted - Actual) ** 2)

In [ ]:
def neural_network(X, Y, num_iterations=10):
    n_x, n_h, n_y = network_architecture(X, Y)

    parameters = define_network_parameters(n_x, n_h, n_y)

    cost_history = [] # To store the cost at each iteration

    for i in range(num_iterations):
      activations = forward_propagation(X, parameters)

      cost = compute_error(activations['A2'], Y) # Assuming you've defined this function
      cost_history.append(cost)

      derivatives = backward_propagation(parameters, activations, X, Y)

      parameters = update_parameters(parameters, derivatives)

    return parameters, cost_history

In [ ]:
model, cost = neural_network(X, Y, num_iterations=200)


plt.plot(cost)
plt.suptitle('Cost History')
plt.xlabel('Iteration')
plt.ylabel('Cost')

In [ ]:
def predict(parameters, X):
    results = forward_propagation(X, parameters)

    predictions = np.around(results['A2'])

    return predictions

    predictions = predict(model, X)

In [ ]:
Error = []

for i in range(len(predictions[0])):
    if predictions[0][i] != Y[0][i]:
        Error.append(i)

print("Number of misclassified examples:", len(Error))
print("Indices of misclassified examples:", Error)

accuracy = ((X.shape[0] - len(Error)) / X.shape[0]) * 100
accuracy = (1 - len(Error) / X.shape[0]) * 100

print("Accuracy: {:f}%".format(accuracy))

In [1]:

fig = plt.figure(figsize=(8, 8))
columns = 4
rows = 10
j = 0
for i in Error:
    if j < columns * rows:
        X1 = X[:, i]  # Extract the i-th column (i-th image)
        X1 = X1.reshape(28, 28)  # Reshape the flattened image to (28, 28)
        fig.add_subplot(rows, columns, j + 1)  # Add a subplot to the figure
        plt.imshow(X1, cmap='gray')  # Display the image using a grayscale colormap
        j = j + 1

KeyboardInterrupt: 